# Inicializar

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from math import sqrt, exp, log, pi


def somatorio(erros, pontos):
  vetor_x = pontos[:, 0]
  return np.sum(vetor_x * erros)

def criar_potencias(x, tamanho):
  # tamanho final: (n_atributos * P) + 1
  lista = [1]
  try:
    x.shape[0] == 0
  except:
    x = np.array([x])
  aux = np.array(x)
  for val in aux:
    for pot in range(1, tamanho):
      lista.append(val ** pot)
  return np.array(lista)

def calcular_MSE(dataset_x, dataset_y, parametros):
  # parametros = (n_atrib * P) + 1
  N = dataset_x.shape[0]
  n_atrib = dataset_x.shape[1]
  P = (parametros.shape[0] - 1) // n_atrib

  y_barra = dataset_x @ parametros.T
  erro = dataset_y - y_barra
  MSE = np.sum(erro**2) / (2*N)
  return MSE

def retorna_o_mesmo(x):
  return x

def func_logistica(x):
  return 1 / (1 + np.exp(-x))

def deriv_func_logistica(x):
  return func_logistica(x) * (1 - func_logistica(x))

def cross_entropy_loss(X, y, w):
  N = X.shape[0]
  soma = 0
  for i in range(0, N):
    soma -= ( y[i] * log( func_logistica( w @ X[i] ) ) + (1 - y[i]) * log( 1 - func_logistica( w @ X[i] ) ) ) / N
  return soma

def cross_entropy_loss_multiclasse(X, y_real, W):
  y_pred = softmax( X @ W.T )
  epsilon = 0.000000001
  N = y_real.shape[0]
  y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
  return - (np.sum(y_real * np.log(y_pred))) / N

def erro_reg_logistica(y, w, X):
  # y = (N, 1)
  # X = (N, M+1)
  # w = (M+1, 1)
  vetorizar = np.vectorize(func_logistica)

  return y - vetorizar( w @ X.T )

def softmax(y_pred):
  exp_pontuacoes = np.exp(y_pred)
  return (exp_pontuacoes /  np.sum(np.atleast_2d(exp_pontuacoes), axis=1, keepdims = True))


def k_fold(X, Y, k):
  n,m = X.shape
  ordenacao = np.arange(n)
  np.random.shuffle(ordenacao)
  saida = []
  intervalo = n // k
  grupos_X = []
  grupos_Y = []
  for i in range(k):
    grupos_X.append( X[ordenacao % k == i] )
    grupos_Y.append( Y[ordenacao % k == i] )

  return grupos_X, grupos_Y


def criar_estatisticas(y_pred, y_real, K):
  iguais = y_pred == y_real
  n = y_real.shape[0]
  acuracia_global = np.sum(iguais) / n
  acuracia_grupos = []
  for i in range(K):
    acuracia_grupos.append( np.sum( iguais[y_real == i]) / np.sum(y_real == i))
  return (acuracia_global, acuracia_grupos)
  


# Normalização

In [2]:
def normalizar(M):
  M_min = np.min(M, axis = 0)
  M_max = np.max(M, axis = 0)
  X = (M - M_min) / (M_max - M_min)
  return (X, M_min, M_max)

def normalizar2(M, M_min, M_max):
  X = (M - M_min) / (M_max - M_min)
  return (X, M_min, M_max)

def desnormalizar(y_normalizado, y_min, y_max):
  return y_normalizado * (y_max - y_min) + y_min

# Regressões

In [3]:
def gd(pontos_x, pontos_y, alfa, qnt_iteracoes, P = 1, hiperparametro = 0, tipo = "linear", plotar = False):
  if tipo not in ["linear", "func_log", "softmax"]:
    raise ("Tipo indefinido, os tipos disponíveis são: linear, func_log, softmax")
  n = pontos_x.shape[0]
  m = pontos_x.shape[1]
  try: K = pontos_y.shape[1]
  except: K = 1

  pontos_y = np.atleast_2d(pontos_y) # (n, K)
  X = np.array([criar_potencias(i,P+1) for i in pontos_x]) # (n, P*m+1)

  estimacoes_y = np.zeros(shape = (n,K)) # (n,K)
  erros_y = np.zeros(shape = (n,K)) # (n,K)
  W = np.ones(shape = (K, P * m + 1)) # (K, P*m+1)
  t = 0

  historico_custo = []
  dict_funcoes = {"linear": retorna_o_mesmo, "func_log": func_logistica, "softmax": softmax}
  dict_custos = {"linear": calcular_MSE, "func_log": cross_entropy_loss, "softmax": cross_entropy_loss_multiclasse}

  f = dict_funcoes[tipo]

  
  for _ in range(qnt_iteracoes):
    t = t + 1

    estimacoes_y = f(X @ W.T) # (n, K)
    erros_y = pontos_y - estimacoes_y

    if plotar:
      custo = dict_custos[tipo]
      historico_custo.append( custo(X, pontos_y, W) )

    if K > 1:
      for k in range(K):
        W[k] = W[k] + alfa * (np.sum((erros_y[:,k] * X.T).T,axis = 0)) / n - hiperparametro * W[k]
    else:
      W = W + alfa * (np.sum((erros_y.T @ X),axis = 0) / n - hiperparametro * W)

  if plotar: 
    plt.plot(range(qnt_iteracoes), historico_custo)
    plt.show()
  return W


def sgd(pontos_x, pontos_y, alfa, qnt_iteracoes, P = 1, hiperparametro = 0, tipo = "linear", plotar = False):
  if tipo not in ["linear", "func_log", "softmax"]:
    raise ("Tipo indefinido, os tipos disponíveis são: linear, func_log, softmax")
  n = pontos_x.shape[0]
  m = pontos_x.shape[1]
  try: K = pontos_y.shape[1]
  except: K = 1
  # pontos_x (n,m)
  # pontos_y (n,K)

  X = np.array([criar_potencias(i,P+1) for i in pontos_x]) # (n, P*m+1)

  estimacoes_y = np.zeros(shape = (n,K)) # (n,K)
  erros_y = np.zeros(shape = (n,K)) # (n,K)
  W = np.ones(shape = (K, P * m + 1)) # (K, P*m+1)
  t = 0

  historico_custo = []
  dict_funcoes = {"linear": retorna_o_mesmo, "func_log": func_logistica, "softmax": softmax}
  dict_custos = {"linear": calcular_MSE, "func_log": cross_entropy_loss, "softmax": cross_entropy_loss_multiclasse}

  f = dict_funcoes[tipo]
  
  for _ in range(qnt_iteracoes):
    for i in range(n):
      t = t + 1
    
      estimacoes_y[i] = f(X[i] @ W.T)
      erros_y[i] = pontos_y[i] - estimacoes_y[i]

      if K > 1:
        for k in range(K):
          W[k] = W[k] + alfa * (erros_y[i,k]/n * X[i] - hiperparametro * W[k])
      else:
        W = W + alfa * (erros_y[i]/n * X[i] - hiperparametro * W)

    if plotar:
      custo = dict_custos[tipo]
      historico_custo.append( custo(X, pontos_y, W) )

  if plotar: 
    plt.plot(range(qnt_iteracoes), historico_custo)
    plt.show()
  return W


def ols(vetor_x, vetor_y, P = 1, hiperparametro = 0):
  m = vetor_x.shape[1]
  n = vetor_x.shape[0]
  X = np.array([criar_potencias(i,P+1) for i in vetor_x]) # (n, P*m + 1)

  W = (np.linalg.inv(X.T @ X + hiperparametro * np.eye((P * m + 1)))) @ X.T @ vetor_y
  
  return W.T

# Classificadores estatísticos

In [4]:
def matriz_desvios(x, media):
  desvio = np.atleast_2d(x-media)
  matriz_dev = desvio.T @ desvio
  return matriz_dev
  
def gerar_mat_cov(X):
  n, p = X.shape
  media = np.mean(X, axis=0)
  vetor_matrizes = np.array([ matriz_desvios(X[i], media) for i in range(n)])
  return np.sum(vetor_matrizes, axis = 0) / (n-1)

def variancia(matriz):
  n,m = matriz.shape
  medias = np.mean(matriz, axis = 0)
  desvios = matriz - medias
  return np.sum(desvios * desvios, axis = 0) / (n - 1)


class Classificadores_Est:
  def __init__(self, dataset_x, dataset_y):
    self.X = dataset_x
    self.Y = dataset_y
    self.N = dataset_x.shape[0]
    try: self.M = dataset_x.shape[1]
    except: self.M = 1
    try: self.K = len(np.unique(dataset_y))
    except: self.K = 1
    self.grupos = [dataset_x[dataset_y == i] for i in np.unique(dataset_y)]
    self.N_classes = np.array([len(i) for i in self.grupos])
    self.medias = np.array([ np.mean(i, axis = 0) for i in self.grupos ])
    self.matriz_covariancia = np.array([ gerar_mat_cov(self.grupos[i]) for i in range(self.K) ])
    self.covariancia_colunas = np.array([ i.diagonal() for i in self.matriz_covariancia]) #!




  def discriminante_gaussiano(self, teste_x, probabilidades = "equiprovaveis"):
    if probabilidades == "equiprovaveis": 
      p = np.ones(self.K) * (1/self.K)

    t_N, t_M = teste_x.shape

    # desvio = (t_N, M)
    # matriz_covariancia = (M, M)

    epsilon = 0.000000001
    saida = []
    for k in range(self.K):
      arg1 = log(np.linalg.det(self.matriz_covariancia[k]) + epsilon) / 2
      desvio = teste_x - self.medias[k]
      arg2 = []
      for i in range(t_N):
        arg2.append((desvio[i] @ np.linalg.inv(self.matriz_covariancia[k] + np.eye(self.M) * epsilon) @ desvio[i].T) / 2)
      arg2 = np.array(arg2)
      arg3 = log(p[k] + epsilon)
      saida.append(- arg1 - arg2 + arg3)
    
    return np.argmax(np.array(saida).T, axis = 1)

  def naive_bayes_gaussiano(self, teste_x, probabilidades = "equiprovaveis"):
    if probabilidades == "equiprovaveis": 
      p = np.ones(self.K) * (1/self.K)

    t_N, t_M = teste_x.shape

    epsilon = 0.000000001
    saida = []
    for k in range(self.K):
      arg1 = log(p[k] + epsilon)
      arg2 = np.sum(np.log(self.covariancia_colunas[k]*2*pi+epsilon)) / 2
      desvio = teste_x - self.medias[k]
      arg3 = np.sum(desvio * desvio / self.covariancia_colunas[k], axis = 1) / 2
      saida.append(arg1 - arg2 - arg3)
    
    return np.argmax(np.array(saida).T, axis = 1)
  
 


# Funcoes para plotagem

In [5]:
def plotar_regressao(dataset, parametros):
  dataset_x = dataset[:,:-1]
  dataset_y = dataset[:,-1]
  min_x = min(dataset_x)
  max_x = max(dataset_x)
  P = parametros.shape[0]
  
  # Parametros (1, P)
  intervalos_x = np.linspace(min_x, max_x, 100) # (1,100)
  X = np.array([criar_potencias(i,P) for i in intervalos_x]) # (100, P)
  intervalos_y = parametros @ X.T

  

  # Plotando os pontos
  plt.plot(dataset_x,dataset_y,'o')

  # Plotando a reta
  plt.plot(intervalos_x,intervalos_y)
  plt.show()

def plotar_regressao_2atributos(dataset, parametros):
  dataset_x = dataset[:,0]
  dataset_y = dataset[:,1]
  min_x = min(dataset_x)
  max_x = max(dataset_x)
  min_y = min(dataset_y)
  max_y = max(dataset_y)
  N = dataset_x.shape[0]
  P = 1

  
  # Parametros (1, P)
  intervalos_x = np.linspace(min_x, max_x, 100) # (1,100)
  X = np.array([criar_potencias(i,P+1) for i in intervalos_x]) # (100, P)
  intervalos_y = X @ parametros[:2]

  intervalos_x = intervalos_x
  intervalos_y = intervalos_y

  # Plotando os pontos
  plt.plot(dataset_x,dataset_y,'o')

  # Plotando a reta
  
  plt.plot(intervalos_x,intervalos_y * (max_y - min_y))
  plt.show()

# Primeira Questão

In [6]:
dataset_q1 = np.genfromtxt('./breastcancer.csv', delimiter=',')
dataset_X = dataset_q1[:,:-1]
dataset_Y = dataset_q1[:,-1]

grupos_X, grupos_Y = k_fold(dataset_X, dataset_Y, 10)
K = len(np.unique(dataset_Y))

# GD
print("------------- GD -------------")
array_acc_global = []
array_acc_grupos = []
for k in range(10):
  treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
  treino_Y = np.atleast_2d(np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])).T
  teste_X = grupos_X[k]
  teste_Y = grupos_Y[k]

  treinoX_norm, min_X, max_X = normalizar(treino_X)
  testeX_norm, *_ = normalizar2(teste_X, min_X, max_X)

  # 0.8
  params = gd(treinoX_norm, treino_Y, 0.8, 2000, tipo = "func_log")

  X = np.array([criar_potencias(i,1+1) for i in testeX_norm])
  predicoes_Y = (func_logistica( X @ params.T ) > 0.5).astype(int)
  predicoes_Y = np.atleast_2d(predicoes_Y)
  teste_Y = np.atleast_2d(teste_Y).T
  iguais = predicoes_Y == teste_Y
  acc_global, acc_grupos = criar_estatisticas(predicoes_Y, teste_Y, K)
  array_acc_global.append(acc_global)
  array_acc_grupos.append(acc_grupos)


media_acc_global = np.mean( np.array(array_acc_global) )
media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_global = np.std( np.array(array_acc_global) )
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)
print("Valor médio acurácia global:", media_acc_global)
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia global:", dp_acc_global)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()


# Discriminante Gaussiano
print("------------- Discriminante Gaussiano -------------")
array_acc_global = []
array_acc_grupos = []
for k in range(10):
  treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
  treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
  teste_X = grupos_X[k]
  teste_Y = grupos_Y[k]
  
  n = teste_X.shape[0]

  modelo = Classificadores_Est(treino_X, treino_Y)
  predicoes_Y = modelo.discriminante_gaussiano(teste_X)
  iguais = predicoes_Y == teste_Y
  acc_global, acc_grupos = criar_estatisticas(predicoes_Y, teste_Y, K)
  array_acc_global.append(acc_global)
  array_acc_grupos.append(acc_grupos)

media_acc_global = np.mean( np.array(array_acc_global) )
media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_global = np.std( np.array(array_acc_global) )
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)
print("Valor médio acurácia global:", media_acc_global)
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia global:", dp_acc_global)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()


# Naive Bayes Gaussiano
print("------------- Naive Bayes Gaussiano -------------")
array_acc_global = []
array_acc_grupos = []
for k in range(10):
  treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
  treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
  teste_X = grupos_X[k]
  teste_Y = grupos_Y[k]
  
  n = teste_X.shape[0]

  modelo = Classificadores_Est(treino_X, treino_Y)
  predicoes_Y = modelo.naive_bayes_gaussiano(teste_X)
  iguais = predicoes_Y == teste_Y
  acc_global, acc_grupos = criar_estatisticas(predicoes_Y, teste_Y, K)
  array_acc_global.append(acc_global)
  array_acc_grupos.append(acc_grupos)

media_acc_global = np.mean( np.array(array_acc_global) )
media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_global = np.std( np.array(array_acc_global) )
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)
print("Valor médio acurácia global:", media_acc_global)
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia global:", dp_acc_global)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()


'''
GD foi o melhor pra predizer as classes considerando a acurácia global, além de ser o melhor modela para 
predizer a primeira classe

Discriminante Gaussiano teve a menor acurácia global, entretanto ele foi o melhor modelo para classficar a
segunda classe

Naive Bayes teve uma boa acurácia global, mas ele não apresenta nenhuma vantagem de acurácia global ou 
acurácia por grupos em comparação aos outros modelos

Uma boa estratégia para esse dataset seria utilizar da Regressão Logística pra a predição da classe 1, e usar o
Discriminante Gaussiano para a predição da classe 2
'''

------------- GD -------------
Valor médio acurácia global: 0.9754385964912281
Valor médio acurácia por grupo: [0.99161765 0.94812522]
Desvio Padrão acurácia global: 0.02246710258748368
Desvio Padrão acurácia por grupo: [0.01839176 0.04497256]

------------- Discriminante Gaussiano -------------
Valor médio acurácia global: 0.875187969924812
Valor médio acurácia por grupo: [0.80399225 0.9962963 ]
Desvio Padrão acurácia global: 0.038904477016405516
Desvio Padrão acurácia por grupo: [0.0652137  0.01111111]

------------- Naive Bayes Gaussiano -------------
Valor médio acurácia global: 0.9350250626566415
Valor médio acurácia por grupo: [0.95738221 0.89635769]
Desvio Padrão acurácia global: 0.037619091449450935
Desvio Padrão acurácia por grupo: [0.02951596 0.07744559]



'\nGD foi o melhor pra predizer as classes considerando a acurácia global, além de ser o melhor modela para \npredizer a primeira classe\n\nDiscriminante Gaussiano teve a menor acurácia global, entretanto ele foi o melhor modelo para classficar a\nsegunda classe\n\nNaive Bayes teve uma boa acurácia global, mas ele não apresenta nenhuma vantagem de acurácia global ou \nacurácia por grupos em comparação aos outros modelos\n\nUma boa estratégia para esse dataset seria utilizar da Regressão Logística pra a predição da classe 1, e usar o\nDiscriminante Gaussiano para a predição da classe 2\n'

# Segunda Questão

In [7]:
dataset_q2 = np.genfromtxt('./vehicle.csv', delimiter=',')
dataset_X = dataset_q2[:,:-1]
dataset_Y = dataset_q2[:,-1]

grupos_X, grupos_Y = k_fold(dataset_X, dataset_Y, 10)
K = len(np.unique(dataset_Y))

# GD
print("------------- GD -------------")
array_acc_global = []
array_acc_grupos = []
for k in range(10):
  treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
  treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:]).astype(int)
  treino_Y = np.array([ np.eye(K)[i] for i in treino_Y ])
  teste_X = grupos_X[k]
  teste_Y = grupos_Y[k].astype(int)

  treinoX_norm, min_X, max_X = normalizar(treino_X)
  testeX_norm, *_ = normalizar2(teste_X, min_X, max_X)

  params = gd(treinoX_norm, treino_Y, 0.8, 6000, tipo = "softmax")

  X = np.array([criar_potencias(i,1+1) for i in testeX_norm])
  predicoes_Y = softmax( X @ params.T )
  predicoes_Y = np.atleast_2d(predicoes_Y)
  predicoes_Y = np.argmax(predicoes_Y,axis = 1)

  teste_Y = np.atleast_2d(teste_Y).astype(int)
  iguais = predicoes_Y == teste_Y
  acc_global, acc_grupos = criar_estatisticas(predicoes_Y, teste_Y, K)
  array_acc_global.append(acc_global)
  array_acc_grupos.append(acc_grupos)


media_acc_global = np.mean( np.array(array_acc_global) )
media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_global = np.std( np.array(array_acc_global) )
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)
print("Valor médio acurácia global:", media_acc_global)
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia global:", dp_acc_global)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()


# Discriminante Gaussiano
print("------------- Discriminante Gaussiano -------------")
array_acc_global = []
array_acc_grupos = []
for k in range(10):
  treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
  treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
  teste_X = grupos_X[k]
  teste_Y = grupos_Y[k]
  
  n = teste_X.shape[0]

  modelo = Classificadores_Est(treino_X, treino_Y)
  predicoes_Y = modelo.discriminante_gaussiano(teste_X)
  iguais = predicoes_Y == teste_Y
  acc_global, acc_grupos = criar_estatisticas(predicoes_Y, teste_Y, K)
  array_acc_global.append(acc_global)
  array_acc_grupos.append(acc_grupos)

media_acc_global = np.mean( np.array(array_acc_global) )
media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_global = np.std( np.array(array_acc_global) )
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)
print("Valor médio acurácia global:", media_acc_global)
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia global:", dp_acc_global)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()


# Naive Bayes Gaussiano
print("------------- Naive Bayes Gaussiano -------------")
array_acc_global = []
array_acc_grupos = []
for k in range(10):
  treino_X = np.concatenate( grupos_X[:k] + grupos_X[k+1:])
  treino_Y = np.concatenate( grupos_Y[:k] + grupos_Y[k+1:])
  teste_X = grupos_X[k]
  teste_Y = grupos_Y[k]
  
  n = teste_X.shape[0]

  modelo = Classificadores_Est(treino_X, treino_Y)
  predicoes_Y = modelo.naive_bayes_gaussiano(teste_X)
  iguais = predicoes_Y == teste_Y
  acc_global, acc_grupos = criar_estatisticas(predicoes_Y, teste_Y, K)
  array_acc_global.append(acc_global)
  array_acc_grupos.append(acc_grupos)

media_acc_global = np.mean( np.array(array_acc_global) )
media_acc_grupos = np.mean( np.array(array_acc_grupos) , axis = 0)
dp_acc_global = np.std( np.array(array_acc_global) )
dp_acc_grupos = np.std( np.array(array_acc_grupos) , axis = 0)
print("Valor médio acurácia global:", media_acc_global)
print("Valor médio acurácia por grupo:",media_acc_grupos)
print("Desvio Padrão acurácia global:", dp_acc_global)
print("Desvio Padrão acurácia por grupo:", dp_acc_grupos)
print()


'''
O modelo GD mostrou ser bem eficiente na classificação da primeira e última classe (porém, menos que o discriminante 
gaussiano), entretanto é importante destacar o seu alto desvio padrão da acurácia global, isso mostra que o modelo 
é muito instável e pode ter resultados bem diferentes dependendo do conjunto de dados

O modelo do Discriminante Gaussiano é sem dúvidas o melhor modelo para este dataset entre os três escolhidos, sua
acurácia global é a maior dos três, além da acurácia de cada grupo também ser maior entre todos os modelos. Além 
disso, ele não possui o problema de instabilidade que o modelo GD possui

o modelo de Naives Bayes Gaussiano teve uma baixíssima acurácia global e foi péssimo em classificar o primeiro grupo,
ele não possui nenhuma vantagem para ser utilizado nesse dataset

Vale destacar que, nesse dataset, os modelos tiveram dificuldade de classificar os grupos 2 e 3, muito provavelmente
porque essas duas classes devem possuir características semelhantes.
'''

------------- GD -------------
Valor médio acurácia global: 65.0
Valor médio acurácia por grupo: [0.91662628 0.57434889 0.63321309 0.97164048]
Desvio Padrão acurácia global: 2.280350850198276
Desvio Padrão acurácia por grupo: [0.05500565 0.11145823 0.12209297 0.02880608]

------------- Discriminante Gaussiano -------------
Valor médio acurácia global: 0.8511484593837535
Valor médio acurácia por grupo: [0.97840909 0.73904816 0.69945526 0.991     ]
Desvio Padrão acurácia global: 0.03707131319395602
Desvio Padrão acurácia por grupo: [0.0350476  0.11177631 0.07351946 0.01813836]

------------- Naive Bayes Gaussiano -------------
Valor médio acurácia global: 0.4588235294117647
Valor médio acurácia por grupo: [0.16727221 0.42770575 0.3867778  0.88599085]
Desvio Padrão acurácia global: 0.06724191592131634
Desvio Padrão acurácia por grupo: [0.06423534 0.0761307  0.10812561 0.07092715]



'\nO modelo GD mostrou ser bem eficiente na classificação da primeira e última classe (porém, menos que o discriminante \ngaussiano), entretanto é importante destacar o seu alto desvio padrão da acurácia global, isso mostra que o modelo \né muito instável e pode ter resultados bem diferentes dependendo do conjunto de dados\n\nO modelo do Discriminante Gaussiano é sem dúvidas o melhor modelo para este dataset entre os três escolhidos, sua\nacurácia global é a maior dos três, além da acurácia de cada grupo também ser maior entre todos os modelos. Além \ndisso, ele não possui o problema de instabilidade que o modelo GD possui\n\no modelo de Naives Bayes Gaussiano teve uma baixíssima acurácia global e foi péssimo em classificar o primeiro grupo,\nele não possui nenhuma vantagem para ser utilizado nesse dataset\n\nVale destacar que, nesse dataset, os modelos tiveram dificuldade de classificar os grupos 2 e 3, muito provavelmente\nporque essas duas classes devem possuir características semel